<a href="https://colab.research.google.com/github/SatadruMukherjee/Data-Preprocessing-Models/blob/main/Using%20KMS%20for%20Client%20Side%20Encryption.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install boto3

In [ ]:
!pip install cryptography

In [9]:
import boto3
import base64
import logging
from cryptography.fernet import Fernet

**Fernet guarantees that a message encrypted using it cannot be manipulated or read without the key.**
**Fernet is an implementation of symmetric (also known as “secret key”) authenticated cryptography.**


In [3]:
aws_access_key="{}"
aws_secret_key="{}"

In [4]:
import boto3
session = boto3.Session(
    aws_access_key_id=aws_access_key,
    aws_secret_access_key=aws_secret_key,
    region_name='{}'
)

In [5]:
kms_client=session.client('kms')

In [6]:
def create_data_key(cmk_id, key_spec='AES_256'):
    """Generate a data key to use when encrypting and decrypting data

    :param cmk_id: KMS CMK ID or ARN under which to generate and encrypt the
    data key.
    :param key_spec: Length of the data encryption key. Supported values:
        'AES_128': Generate a 128-bit symmetric key
        'AES_256': Generate a 256-bit symmetric key
    :return Tuple(EncryptedDataKey, PlaintextDataKey) where:
        EncryptedDataKey: Encrypted CiphertextBlob data key as binary string
        PlaintextDataKey: Plaintext base64-encoded data key as binary string
    :return Tuple(None, None) if error
    """
    try:
        response = kms_client.generate_data_key(KeyId=cmk_id, KeySpec=key_spec)
    except ClientError as e:
        logging.error(e)
        return None, None

    # Return the encrypted and plaintext data key
    return response['CiphertextBlob'], base64.b64encode(response['Plaintext']).decode(encoding="utf-8")

In [10]:
ms=create_data_key('{}', key_spec='AES_256')

In [ ]:
data_key=ms[1]
print(data_key)

In [ ]:
absolute_file_path_to_be_encrypted=input('Enter the filepath : ');
encrpted_file_name_with_path=input('Enter the output filepath : ');

f = Fernet(data_key)

with open(absolute_file_path_to_be_encrypted, 'rb') as original_file:
    original = original_file.read()

encrypted = f.encrypt(original)

with open (encrpted_file_name_with_path, 'wb') as encrypted_file:
    encrypted_file.write(encrypted)

In [14]:
with open(encrpted_file_name_with_path, 'rb') as encrypted_file:
    encrypted = encrypted_file.read()

decrypted = f.decrypt(encrypted)

with open('testingdecode.txt', 'wb') as decrypted_file:
    decrypted_file.write(decrypted)